## Error estimate
We use a known function (see uref below) to confirm the theorical error estimate.

For each couple $(k,d)$ there are four tests (Essential/Natural Normal/Tangential boundary condition)
We aim to test for polynomial degree below 3 and for $k = (0,1,2)$ ($k$ define the frequency of uref)

compute_slope give 3 lists of 2 sequence, these two compute the slope as a function of hmin and of hmax.
Among the 3, one is the error on $u$, and the last two are errors on $curl(u)$ projected on $F0$ or $F2$.
While $F2$ seem natural, it actually give worse result than $F0$. (This amount to a greater distance between urefcurl and the discret space $F2$ (DG) than between urefcurl and $F0$ (P))

checklist :

k = 0, deg = 1

k = 0, deg = 2

k = 1, deg = 2

k = 1, deg = 1

In [1]:
import os, sys, inspect
os.environ["OPENBLAS_NUM_THREADS"] = "4"
k = 0.0
deg = 2
Elemdict = {
        '0f' : {'form' : 'trimmed', 'degree' : deg}, 
        '1f' : {'form' : 'trimmed', 'degree' : deg}, 
        '2f' : {'form' : 'trimmed', 'degree' : deg}}
PREFIX = "Conv_harmonics/"

sys.path.insert(0,os.path.realpath(os.path.join(os.path.abspath(os.path.split(inspect.getfile( inspect.currentframe() ))[0]),'..','pymodule')))

In [2]:
import matplotlib.pyplot as plt
from dolfin import *
import numpy as np
from mshr import *
from BTsolver_2D import BiotSavart_harmonic
import javabutton as js
import math

In [3]:
def curl2d(u):
    return u[1].dx(0) - u[0].dx(1)

def compute_error(T,ref,refcurl,mesh,k,expected_harmonics=0,customthreshold=1e-15,Elemdict=None,savefile=False,degree="",kprint="",fileid=""):
    #mesh = generate_mesh(geo,k)
    zeroexp = Expression("0.",degree=2)
    if (T == "NatNor"):
        f0 = zeroexp
        f2 = refcurl
        adj = False
        dbc = False
    elif(T == "NatTan"):
        f0 = refcurl
        f2 = zeroexp
        adj = True
        dbc = False
    elif (T == "EssTan"):
        f0 = zeroexp
        f2 = refcurl
        adj = False
        dbc = True
    elif(T == "EssNor"):
        f0 = refcurl
        f2 = zeroexp
        adj = True
        dbc = True
    else:
        print("Valid value are : NatNor NatTan EssTan EssNor")
        return["hmin","hmax","erroru","errordu0","errordu2"]
    have_1_harmonics = (expected_harmonics > 0)
    if Elemdict is not None:
        solver = BiotSavart_harmonic(DBC=dbc,Elemdict=Elemdict)
    else:
        solver = BiotSavart_harmonic(DBC=dbc)
    solver.init_mesh(mesh,search_harmonics=have_1_harmonics,expected_harmonics=expected_harmonics,customthreshold=customthreshold)
    solver.fe0 = f0
    solver.fe2 = f2
    solver.interpolate()
    if(adj):
        B = solver.solve_1_form_dual()
    else:
        usolv = solver.solve()
        B = usolv.split(True)[1]
    erroru = errornorm(ref,B)
    errordu0 = errornorm(refcurl,project(curl2d(B),solver.F0))
    errordu2 = errornorm(refcurl,project(curl2d(B),solver.F2))
    if(savefile):
        fig = plt.figure(figsize=(20,20))
        plt.subplot(221)
        plt.title("Reference function")
        p = plot(ref,mesh=mesh)
        plt.colorbar(p)
        plt.subplot(222)
        plt.title("Solution function")
        p = plot(B)
        plt.colorbar(p)
        plt.subplot(223)
        plt.title("Reference curl")
        p = plot(refcurl,mesh=mesh)
        plt.colorbar(p)
        plt.subplot(224)
        plt.title("Solution curl")
        p = plot(curl2d(B))
        plt.colorbar(p)
        with open(PREFIX+"2D_Error_conv_"+str(kprint)+"_"+str(degree)+"_"+str(T)+"_"+str(k)+"_"+fileid+".png", 'wb') as outfile:
            plt.savefig(outfile)
        plt.close(fig)
    return [mesh.hmin(),mesh.hmax(),erroru,errordu0,errordu2]

def compute_error_betweenspace(ref,refcurl,mesh,Elemdict = None):
    # mesh = generate_mesh(geo,k)
    if Elemdict is not None:
        solver = BiotSavart_harmonic(Elemdict = Elemdict)
    else:
        solver = BiotSavart_harmonic()
    solver.init_mesh(mesh,search_harmonics=False)
    erroru = errornorm(ref,project(ref,solver.F1))
    errordu0 = errornorm(refcurl,project(refcurl,solver.F0))
    errordu2 = errornorm(refcurl,project(refcurl,solver.F2))
    
    return [mesh.hmin(),mesh.hmax(),erroru,errordu0,errordu2]

def compute_slope(L):
    '''
    Input L : [hmin,hmax,erru,errdu0,errdu2]
    Ouput S : [[ln(erru)/ln(hmin),ln(erru)/ln(hmax)],[ln(errdu0)/ln(hmin),ln(errdu0)/ln(hmax)],[ln(errdu2)/ln(hmin),ln(errdu2)/ln(hmax)]]
    '''
    Slope = [[[float('nan'),float('nan')]],[[float('nan'),float('nan')]],[[float('nan'),float('nan')]]]
    for i in range(1,len(L)):
        Slope[0].append([(np.log(L[i][2])-np.log(L[i-1][2]))/(np.log(L[i][0])-np.log(L[i-1][0])),(np.log(L[i][2])-np.log(L[i-1][2]))/(np.log(L[i][1])-np.log(L[i-1][1]))])
        Slope[1].append([(np.log(L[i][3])-np.log(L[i-1][3]))/(np.log(L[i][0])-np.log(L[i-1][0])),(np.log(L[i][3])-np.log(L[i-1][3]))/(np.log(L[i][1])-np.log(L[i-1][1]))])
        Slope[2].append([(np.log(L[i][4])-np.log(L[i-1][4]))/(np.log(L[i][0])-np.log(L[i-1][0])),(np.log(L[i][4])-np.log(L[i-1][4]))/(np.log(L[i][1])-np.log(L[i-1][1]))])
    return Slope
def save_slope(L,T,degree,kprint,fileid=""):
    S = compute_slope(L)
    with open(PREFIX+"2D_Error_conv_"+str(kprint)+"_"+str(degree)+"_"+str(T)+"_"+fileid+".dat", 'w') as outfile:
        outfile.write("#{0:>19s},{1:>20s},{2:>20s},{3:>20s},{4:>20s},{5:>20s},{6:>20s},{7:>20s},{8:>20s},{9:>20s},{10:>20s}\n".format(
                "hmin", "hmax", "Rate erru/hmin", "Rate errdu0/hmin", "Rate errdu2/hmin",  
                "Rate erru/hmax", "Rate errdu0/max", "Rate errdu2/max", 
                "erru", "errdu0", "errdu2"))
        #outfile.write("#{0:19.4f},{1:20.4f},{2:20.4f},{3:20.4f},{4:20.4f},{5:20.4f},{6:20.4f},{7:20.4f},{8:20.4f},{9:20.4f},{10:20.4f}\n".format(
        #        L[0][0],L[0][1],
        #        S[0][0][0],S[1][0][0],S[2][0][0],
        #        S[0][0][1],S[1][0][1],S[2][0][1],
        #        L[0][2],L[0][3],L[0][4]))
        for i in range(len(L)):
            outfile.write("{0:20.14f},{1:20.14f},{2:20.14f},{3:20.14f},{4:20.14f},{5:20.14f},{6:20.14f},{7:20.14f},{8:20.14f},{9:20.14f},{10:20.14f}\n".format(
                L[i][0],L[i][1],
                S[0][i][0],S[1][i][0],S[2][i][0],
                S[0][i][1],S[1][i][1],S[2][i][1],
                L[i][2],L[i][3],L[i][4]))

js.insertButtonCode()

In [4]:
rectangle1 = Rectangle(dolfin.Point(0.0, -0.5),dolfin.Point(1.0, 0.5))
rectangle2 = Rectangle(dolfin.Point(-1.0, -1.5),dolfin.Point(2.0, 1.5))
rectangle1t = Rectangle(dolfin.Point(-0.5, 0.0),dolfin.Point(0.5, 1.0))
rectangle2t = Rectangle(dolfin.Point(-1.5, -1.0),dolfin.Point(1.5, 2.0))

In [5]:
uref = Expression(("sin(k*pi*x[0])*sin(k*pi*x[1])","cos(k*pi*x[0])*cos(k*pi*x[1])"),k=2.*k+1.,pi=math.pi,degree=4)
urefcurl = Expression("-2.*k*pi*sin(k*pi*x[0])*cos(k*pi*x[1])",k=2.*k+1.,pi=math.pi,degree=6)

In [6]:
nbitt = 5
LNatNorErr = []
#mesh = generate_mesh(rectangle2 - rectangle1,4)
mesh = generate_mesh(rectangle2,10)
for i in range(nbitt):
    LNatNorErr.append(compute_error("NatNor",uref,urefcurl,mesh,i,expected_harmonics=0,customthreshold=1e-6,savefile=False,degree=deg,kprint=k,Elemdict=Elemdict))
    mesh = refine(mesh)
LNatTanErr = []
#mesh = generate_mesh(rectangle2t - rectangle1t,4)
mesh = generate_mesh(rectangle2t,10)
for i in range(nbitt):
    LNatTanErr.append(compute_error("NatTan",uref,urefcurl,mesh,i,expected_harmonics=0,customthreshold=1e-6,savefile=False,degree=deg,kprint=k,Elemdict=Elemdict))
    mesh = refine(mesh)
LEssNorErr = []
#mesh = generate_mesh(rectangle2 - rectangle1,4)
mesh = generate_mesh(rectangle2,10)
for i in range(nbitt):
    LEssNorErr.append(compute_error("EssNor",uref,urefcurl,mesh,i,expected_harmonics=0,customthreshold=1e-6,savefile=False,degree=deg,kprint=k,Elemdict=Elemdict))
    mesh = refine(mesh)
LEssTanErr = []
#mesh = generate_mesh(rectangle2t - rectangle1t,4)
mesh = generate_mesh(rectangle2t,10)
for i in range(nbitt):
    LEssTanErr.append(compute_error("EssTan",uref,urefcurl,mesh,i,expected_harmonics=0,customthreshold=1e-6,savefile=False,degree=deg,kprint=k,Elemdict=Elemdict))
    mesh = refine(mesh)


In [7]:
save_slope(LNatNorErr,"NatNor",degree=deg,kprint=k)
save_slope(LNatTanErr,"NatTan",degree=deg,kprint=k)
save_slope(LEssNorErr,"EssNor",degree=deg,kprint=k)
save_slope(LEssTanErr,"EssTan",degree=deg,kprint=k)